# Geoapify Places 5 Cities

In [46]:
# Dependencies
from census import Census
import hvplot.pandas
import time
import requests
import json
import pandas as pd
import numpy as np
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import the API key
from config_key import geoapify_key
from config_key import census_key

In [47]:
#c.acs5.tables()

In [48]:
# Create an instance of the Census library
# 
c = Census(
    census_key, 
    year=2021
)

# Run Census Search to retrieve data on all zip codes (2021 ACS5 Census)
census_data = c.acs5.get(
    (
        "B01003_001E",
        "B17001_002E"
    ),
    {'for': 'zip code tabulation area:*'}
)

# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Column renaming
census_df = census_df.rename(
    columns = {
        "B01003_001E": "Population",
        "B17001_002E": "Poverty Count",
        "zip code tabulation area": "Zipcode",
        #rural vs urban,
        #latitude and latitude
    }
)

In [49]:
census_df

,Population,Poverty Count,Zipcode
0,17126.0,11302.0,00601
1,37895.0,17121.0,00602
2,49136.0,23617.0,00603
3,5751.0,3139.0,00606
4,26153.0,11640.0,00610
...,...,...,...
33769,13.0,0.0,99923
33770,917.0,182.0,99925
33771,1445.0,252.0,99926
33772,11.0,0.0,99927


In [50]:
#batch API call
#url = "https://api.geoapify.com/v1/batch?apiKey=geoapify_key
#headers = {"Content-Type": "application/json"}
#data = '{"api":"/v2/places","params":{},"inputs":[{"id":"optional-input-id1","params":{}}]}'
      
#try:
#    resp = requests.post(url, headers=headers, data=data)
#    print(resp.json())
#except requests.exceptions.HTTPError as e:
#    print (e.response.text)

In [103]:
cities = ["Bronx, NY","Napa, CA", "Victorville, CA","San Francisco, CA"]

places = ["512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205",
          "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208",
          "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208",
          "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"]

place_Bronx = "512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205"
place_Napa = "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208"
place_Victorville = "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208"
place_SanFran = "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"
        

In [104]:
for place in places:
    filters = f"place:{place}"
    categories = "commercial.supermarket"
    radius = 16100 # 10 miles

    params = {
        "categories":categories,
        "apiKey":geoapify_key,
        "filter":filters #,
    }

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    response = requests.get(base_url,params=params)

    print(response.url)    
    

https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205
https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208
https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208
https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208


In [51]:
# Create DataFrame called US_locations_df to store the city, coordinates, supermarket property, distance
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205&limit=100&apiKey=geoapify_key
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208&limit=100&apiKey=geoapify_key
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208&limit=100&apiKey=geoapify_key
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208&limit=100&apiKey=geoapify_key

cities = ["Bronx, NY","Napa, CA", "Victorville, CA","San Francisco, CA"]

places = ["512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205",
          "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208",
          "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208",
          "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"]

place_Bronx = "512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205"
place_Napa = "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208"
place_Victorville = "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208"
place_SanFran = "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"
        


# Set the parameters to search for supermarkets 

#latitude = [40.837048,38.297539,34.536217,37.828724,44.8485]
#longitude = [-73.865433,-122.286865,-117.292763,-122.355537,-123.2340]
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
#radius = 16100 # 10 miles
#radius = 1610 # 1 mile
# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
#filters = f"circle:{longitude},{latitude},{radius}"

filters = f"place:{places[2]}"
categories = "commercial.supermarket"
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
radius = 16100 # 10 miles
#radius = 1610 # 1 mile
#bias = f"proximity:{longitude},{latitude}"
#limit = 100


# Set up a params dictionary
params = {
    "categories":categories,
    "apiKey":geoapify_key,
#    "limit": limit,
    "filter":filters #,
#    "bias":bias
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Make an API request using the params dictionaty
response = requests.get(base_url,params=params)

# print the response url, avoid doing for public github repos in order to avoid exposing key
print(response.url)    

https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208


In [52]:
# convert response to json
supermarket_data = response.json()

# Print the json (pretty printed)
print(json.dumps(supermarket_data, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -117.39681949157831,
                    34.50927068192078
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Walmart Supercenter",
                "address_line2": "12234 Palmdale Road, Victorville, CA 92392, United States of America",
                "categories": [
                    "building",
                    "building.commercial",
                    "commercial",
                    "commercial.supermarket"
                ],
                "city": "Victorville",
                "country": "United States",
                "country_code": "us",
                "county": "San Bernardino County",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
           

In [53]:
# Create DataFrame called US_locations_df to store the city, coordinates, supermarket property, distance
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205&limit=100&apiKey=YOUR_API_KEY
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208&limit=100&apiKey=YOUR_API_KEY
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208&limit=100&apiKey=YOUR_API_KEY
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208&limit=100&apiKey=YOUR_API_KEY

cities = ["Bronx, NY","Napa, CA", "Victorville, CA","San Francisco, CA"]

places = ["512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205",
          "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208",
          "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208",
          "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"]

place_Bronx = "512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205"
place_Napa = "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208"
place_Victorville = "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208"
place_SanFran = "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"
          
#latitude = [40.837048,38.297539,34.536217,37.828724,44.8485]
#longitude = [-73.865433,-122.286865,-117.292763,-122.355537,-123.2340]


# Set the parameters to search for supermarkets 
categories = "commercial.supermarket"
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
#radius = 16100 # 10 miles
#radius = 1610 # 1 mile
i = 0

# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
#filters = f"circle:{longitude},{latitude},{radius}"

filters = f"place:{places[2]}"
categories = "commercial.supermarket"
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
radius = 16100 # 10 miles
#radius = 1610 # 1 mile
#bias = f"proximity:{longitude},{latitude}"
#limit = 100


# Set up a params dictionary
params = {
    "categories":categories,
    "apiKey":geoapify_key,
#    "limit": limit,
    "filter":filters #,
#    "bias":bias
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Make an API request using the params dictionaty
response = requests.get(base_url,params=params)

# print the response url, avoid doing for public github repos in order to avoid exposing key
print(response.url)    

https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208


In [54]:
# convert response to json
supermarket_data = response.json()

# Print the json (pretty printed)
print(json.dumps(supermarket_data, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -117.39681949157831,
                    34.50927068192078
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Walmart Supercenter",
                "address_line2": "12234 Palmdale Road, Victorville, CA 92392, United States of America",
                "categories": [
                    "building",
                    "building.commercial",
                    "commercial",
                    "commercial.supermarket"
                ],
                "city": "Victorville",
                "country": "United States",
                "country_code": "us",
                "county": "San Bernardino County",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
           

In [75]:
# Extracting 'features' and 'properties' from JSON
features = supermarket_data.get("features", [])
rows = [] # Iterate through the results 

In [76]:

# Extracting results from JSON 
# results = supermarket_data.get("results", []) # Create a list to store rows 
supermarket_count = 1

for feature in features: 
    properties = feature.get("properties",{})
    # Extract relevant information
    name = properties.get("name") 
    lat = properties.get("lat") 
    lon = properties.get("lon") # Append data to the list
    postcode = properties.get("postcode")
    radius = properties.get("radius")
    city = properties.get("city")
    rows.append({"Name": name, "City": city, "Zipcode": postcode, "Latitude": lat, "Longitude": lon}) 
    # rows.append({"Name": name, "Latitude": lat, "Longitude": lon, "Supermarket Count": supermarket_count}) 
    # rows.append({"Name": name, "City": city, "Zipcode": postcode, "Latitude": lat, "Longitude": lon, "Supermarket Count": supermarket_count}) 
    #rows.append({"City": city, "Zipcode": postcode, "Supermarket Count": supermarket_count}) 

    
print(rows)

[{'Name': 'Walmart Supercenter', 'City': 'Victorville', 'Zipcode': '92392', 'Latitude': 34.50943605, 'Longitude': -117.39677417201278}, {'Name': 'Walmart Supercenter', 'City': 'Victorville', 'Zipcode': '92392', 'Latitude': 34.465300049999996, 'Longitude': -117.35445325404288}, {'Name': 'WinCo Foods', 'City': 'Victorville', 'Zipcode': '93533', 'Latitude': 34.523200450000004, 'Longitude': -117.32938285894309}, {'Name': 'Vons', 'City': 'Victorville', 'Zipcode': '92392', 'Latitude': 34.4725722, 'Longitude': -117.28965071429727}, {'Name': 'Food 4 Less', 'City': 'Victorville', 'Zipcode': '92392', 'Latitude': 34.51939, 'Longitude': -117.3145208}, {'Name': 'Costco', 'City': 'Victorville', 'Zipcode': '92395', 'Latitude': 34.51486405, 'Longitude': -117.3186445374019}, {'Name': 'Vallarta', 'City': 'Victorville', 'Zipcode': '92392', 'Latitude': 34.47515355, 'Longitude': -117.33935072718029}, {'Name': 'Stater Bros.', 'City': 'Victorville', 'Zipcode': '92392', 'Latitude': 34.47184, 'Longitude': -117

In [77]:
# Create a DataFrame 
supermarket_df = pd.DataFrame(rows) # Display the DataFrame 

# should not return city other than Napa
supermarket_df

,Name,City,Zipcode,Latitude,Longitude
0,Walmart Supercenter,Victorville,92392,34.509436,-117.396774
1,Walmart Supercenter,Victorville,92392,34.465300,-117.354453
2,WinCo Foods,Victorville,93533,34.523200,-117.329383
3,Vons,Victorville,92392,34.472572,-117.289651
4,Food 4 Less,Victorville,92392,34.519390,-117.314521
5,Costco,Victorville,92395,34.514864,-117.318645
6,Vallarta,Victorville,92392,34.475154,-117.339351
7,Stater Bros.,Victorville,92392,34.471840,-117.362971
8,Albertsons,Victorville,92392,34.471805,-117.365750
9,Stater Bros.,Victorville,93533,34.520182,-117.331506


In [78]:
supermarket_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       10 non-null     object 
 1   City       10 non-null     object 
 2   Zipcode    10 non-null     object 
 3   Latitude   10 non-null     float64
 4   Longitude  10 non-null     float64
dtypes: float64(2), object(3)
memory usage: 528.0+ bytes


In [79]:
supermarket_df.columns

Index(['Name', 'City', 'Zipcode', 'Latitude', 'Longitude'], dtype='object')

In [80]:
supermarket_totals_df = supermarket_df.groupby(['City', 'Zipcode']).size().reset_index(name='Supermarket Count')
# supermarket_totals_df = pd.DataFrame[{"City":supermarket_df["City"],"Zipcode":supermarket_df["Zipcode"],"Supermarket Count":supermarket_df["Supermarket Count"]}].groupby["Zipcode"]
supermarket_totals_df

,City,Zipcode,Supermarket Count
0,Victorville,92392,7
1,Victorville,92395,1
2,Victorville,93533,2


In [96]:
census_df

,Population,Poverty Count,Zipcode
0,17126.0,11302.0,00601
1,37895.0,17121.0,00602
2,49136.0,23617.0,00603
3,5751.0,3139.0,00606
4,26153.0,11640.0,00610
...,...,...,...
33769,13.0,0.0,99923
33770,917.0,182.0,99925
33771,1445.0,252.0,99926
33772,11.0,0.0,99927


In [97]:
census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33774 entries, 0 to 33773
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Population     33774 non-null  float64
 1   Poverty Count  33774 non-null  float64
 2   Zipcode        33774 non-null  object 
dtypes: float64(2), object(1)
memory usage: 791.7+ KB


In [82]:
# Merge datasets on Zipcodes
supermarkets_density_final = pd.merge(
    supermarket_totals_df,
    census_df,
    how = "left",
    on = ["Zipcode", "Zipcode"]
)

In [92]:
census_df[census_df['Zipcode']=='92392']

,Population,Poverty Count,Zipcode
31126,61014.0,8939.0,92392


In [99]:
supermarkets_density_final = supermarkets_density_final.fillna(0)

In [101]:
supermarkets_density_final = supermarkets_density_final[["Population","Poverty Count"]].astype(int)

In [102]:
# Add a Supermarket Count column (Supermarkets/Population)
Supermarket_Total = supermarkets_density_final["Supermarket Count"].sum().astype(int)
Population_Total = supermarkets_density_final["Population"].astype(int).groupby(["Zipcode"])
Supermarket_Density = Supermarket_Total / Population_Total

# Configure the final DataFrame
supermarkets_density_final = supermarkets_density_final[
    [
        "postcode",
        "population",
        "supermarket"
    ]
]

# Display DataFrame length and sample data
# print(f"Number of rows in the DataFrame: {len(supermarkets_density_final)}")
supermarkets_density_final.head()

KeyError: 'Supermarket Count'